In [1]:
!wget https://raw.githubusercontent.com/AridHasan/blp25_task1/refs/heads/main/data/subtask_1C/blp25_hatespeech_subtask_1C_train.tsv
!wget https://raw.githubusercontent.com/AridHasan/blp25_task1/refs/heads/main/data/subtask_1C/blp25_hatespeech_subtask_1C_dev.tsv
!wget https://raw.githubusercontent.com/AridHasan/blp25_task1/refs/heads/main/data/subtask_1C/blp25_hatespeech_subtask_1C_test.tsv

--2025-09-13 19:17:01--  https://raw.githubusercontent.com/AridHasan/blp25_task1/refs/heads/main/data/subtask_1C/blp25_hatespeech_subtask_1C_train.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8686966 (8.3M) [application/octet-stream]
Saving to: ‘blp25_hatespeech_subtask_1C_train.tsv’

blp25_hatespeech_su 100%[===================>]   8.28M  --.-KB/s    in 0.08s   

2025-09-13 19:17:01 (106 MB/s) - ‘blp25_hatespeech_subtask_1C_train.tsv’ saved [8686966/8686966]

--2025-09-13 19:17:01--  https://raw.githubusercontent.com/AridHasan/blp25_task1/refs/heads/main/data/subtask_1C/blp25_hatespeech_subtask_1C_dev.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubu

In [2]:
# ✅ Known-good versions for Colab/Py3.12
!pip -q install -U "transformers" "tokenizers" "datasets" "evaluate" "accelerate" "safetensors"

import random, numpy as np, torch
SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
device = "cuda" if torch.cuda.is_available() else "cpu"
device


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.5 MB/s eta 0:00:00


'cuda'

In [3]:
import os, json, logging, numpy as np, pandas as pd, torch, torch.nn as nn
from typing import Dict
from dataclasses import dataclass

from torch.utils.data import DataLoader
from torch.utils.data import RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler

import datasets
from datasets import Dataset, DatasetDict
import evaluate
import transformers
from transformers import (
    AutoTokenizer, AutoConfig,
    AutoModelForSequenceClassification,
    Trainer, TrainingArguments, PreTrainedModel,
    DataCollatorWithPadding
)

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("mtl")

# === Model & tokenization ===
model_name = "xlm-roberta-base"
max_length = 256
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

# === File paths (update if needed) ===
TRAIN_TSV = "/content/blp25_hatespeech_subtask_1C_train.tsv"
DEV_TSV   = "/content/blp25_hatespeech_subtask_1C_dev.tsv"
TEST_TSV  = "/content/blp25_hatespeech_subtask_1C_test.tsv"

assert os.path.exists(TRAIN_TSV), f"Train file not found at {TRAIN_TSV}"
assert os.path.exists(DEV_TSV),   f"Dev file not found at {DEV_TSV}"
assert os.path.exists(TEST_TSV),  f"Test file not found at {TEST_TSV}"

TASKS = ["hate_type", "hate_severity", "to_whom"]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

In [4]:
train_df = pd.read_csv(TRAIN_TSV, sep="\t", dtype=str).fillna("")
dev_df   = pd.read_csv(DEV_TSV,   sep="\t", dtype=str).fillna("")
test_df  = pd.read_csv(TEST_TSV,  sep="\t", dtype=str).fillna("")

print("Train/Dev/Test:", train_df.shape, dev_df.shape, test_df.shape)
print("Columns:", train_df.columns.tolist())

label_maps: Dict[str, dict] = {}
proc_frames: Dict[str, dict] = {}

for task in TASKS:
    uniq = sorted(train_df[task].unique())
    label2id = {lbl: i for i, lbl in enumerate(uniq)}
    id2label = {i: lbl for lbl, i in label2id.items()}
    label_maps[task] = {"label2id": label2id, "id2label": id2label, "num_labels": len(uniq)}

    tr = train_df[["text", task]].copy()
    dv = dev_df[["text", task]].copy()
    te = test_df[["text"]].copy()   # test file usually has no labels

    tr["label"] = tr[task].map(label2id).astype(int)
    dv = dv[dv[task].isin(label2id.keys())].copy()
    dv["label"] = dv[task].map(label2id).astype(int)

    proc_frames[task] = {
        "train": tr[["text", "label"]].rename(columns={"text": "sentence"}),
        "validation": dv[["text", "label"]].rename(columns={"text": "sentence"}),
        "test": te.rename(columns={"text": "sentence"})  # no labels in test
    }

{t: (len(proc_frames[t]["train"]),
     len(proc_frames[t]["validation"]),
     len(proc_frames[t]["test"])) for t in TASKS}

Train/Dev/Test: (35522, 5) (2512, 5) (10200, 2)
Columns: ['id', 'text', 'hate_type', 'hate_severity', 'to_whom']


{'hate_type': (35522, 2512, 10200),
 'hate_severity': (35522, 2512, 10200),
 'to_whom': (35522, 2512, 10200)}

In [6]:
def tokenize_fn(batch):
    return tokenizer(batch["sentence"], padding=False, truncation=True, max_length=max_length)

features = {}
for task in TASKS:
    ds_train = Dataset.from_pandas(proc_frames[task]["train"], preserve_index=False)
    ds_dev   = Dataset.from_pandas(proc_frames[task]["validation"], preserve_index=False)
    ds_test  = Dataset.from_pandas(proc_frames[task]["test"], preserve_index=False)

    raw = DatasetDict({
        "train": ds_train,
        "validation": ds_dev,
        "test": ds_test
    })

    tok = raw.map(tokenize_fn, batched=True, remove_columns=["sentence"])

    # Rename only where the column exists (train/dev), skip test
    for split in ["train", "validation", "test"]:
        if "label" in tok[split].column_names:
            tok[split] = tok[split].rename_column("label", "labels")

    features[task] = tok

features["hate_type"]

Map:   0%|          | 0/35522 [00:00<?, ? examples/s]

Map:   0%|          | 0/2512 [00:00<?, ? examples/s]

Map:   0%|          | 0/10200 [00:00<?, ? examples/s]

Map:   0%|          | 0/35522 [00:00<?, ? examples/s]

Map:   0%|          | 0/2512 [00:00<?, ? examples/s]

Map:   0%|          | 0/10200 [00:00<?, ? examples/s]

Map:   0%|          | 0/35522 [00:00<?, ? examples/s]

Map:   0%|          | 0/2512 [00:00<?, ? examples/s]

Map:   0%|          | 0/10200 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 35522
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 2512
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 10200
    })
})

In [7]:
features["hate_severity"]

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 35522
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 2512
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 10200
    })
})

In [8]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

class StrIgnoreDevice(str):
    def to(self, device): return self

class DataLoaderWithTaskname:
    """Wrap a DataLoader so each batch carries the 'task_name' string."""
    def __init__(self, task_name, data_loader):
        self.task_name = task_name
        self.data_loader = data_loader
        self.batch_size = data_loader.batch_size
        self.dataset = data_loader.dataset
    def __len__(self): return len(self.data_loader)
    def __iter__(self):
        for b in self.data_loader:
            b["task_name"] = StrIgnoreDevice(self.task_name)
            yield b

class MultitaskDataloader:
    """Size-proportional sampling across task loaders (matches your logic)."""
    def __init__(self, dls: Dict[str, DataLoader]):
        self.dls = dls
        self._sizes = {t: len(dl) for t, dl in dls.items()}
        self.task_names = list(dls)
        self.dataset = [None] * sum(len(dl.dataset) for dl in dls.values())
    def __len__(self): return sum(self._sizes.values())
    def __iter__(self):
        choices = []
        for i, t in enumerate(self.task_names):
            choices += [i] * self._sizes[t]
        rng = np.random.default_rng()
        rng.shuffle(choices)
        iters = {t: iter(dl) for t, dl in self.dls.items()}
        for ci in choices:
            t = self.task_names[ci]
            yield next(iters[t])

def _dist_ready():
    return torch.distributed.is_available() and torch.distributed.is_initialized()


In [14]:
class MultitaskModel(transformers.PreTrainedModel):
    def __init__(self, encoder, taskmodels_dict):
        super().__init__(transformers.PretrainedConfig())
        self.encoder = encoder
        self.taskmodels_dict = nn.ModuleDict(taskmodels_dict)

    @classmethod
    def get_encoder_attr_name(cls, model):
        n = model.__class__.__name__
        if n.startswith("Bert"):
            return "bert"
        if n.startswith("Roberta"):
            return "roberta"
        if n.startswith("XLMRoberta"):
            return "roberta"  # XLM-R uses Roberta modules
        if n.startswith("Albert"):
            return "albert"
        if "Indic" in n or "INDIC" in n or "IndicBert" in n:
            # IndicBERT/Indic-MLM models are based on BERT
            return "bert"
        raise KeyError(f"Add support for new model {n}")

    @classmethod
    def create(cls, model_name, model_type_dict: Dict[str, type], model_config_dict: Dict[str, AutoConfig]):
        # Build one head per task, with a shared encoder
        heads = {}
        base = None
        for task, model_type in model_type_dict.items():
            m = model_type.from_pretrained(model_name, config=model_config_dict[task])
            if base is None:
                base = m
            heads[task] = m

        # Pull out the shared encoder from the base model and assign to all heads
        enc_attr = cls.get_encoder_attr_name(base)
        shared_encoder = getattr(base, enc_attr)
        for h in heads.values():
            setattr(h, enc_attr, shared_encoder)

        return cls(encoder=shared_encoder, taskmodels_dict=heads)

    def forward(self, num_items_in_batch=None, **batch):
        task_name: str = batch.pop("task_name")
        return self.taskmodels_dict[task_name](**batch)


# Build configs/heads
configs = {
    t: AutoConfig.from_pretrained(
        model_name,
        num_labels=label_maps[t]["num_labels"],
        id2label=label_maps[t]["id2label"],
        label2id=label_maps[t]["label2id"],
    )
    for t in TASKS
}

multitask_model = MultitaskModel.create(
    model_name=model_name,
    model_type_dict={t: AutoModelForSequenceClassification for t in TASKS},
    model_config_dict=configs,
)

print("Shared encoder OK:", hasattr(multitask_model, "encoder"))

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You sho

Shared encoder OK: True


In [15]:
# If needed at top of the cell:
# import numpy as np

class MultitaskTrainer(transformers.Trainer):
    def get_single_train_dataloader(self, task_name, train_dataset):
        if train_dataset is None:
            raise ValueError("Trainer: training requires a train_dataset.")
        sampler = DistributedSampler(train_dataset) if _dist_ready() else RandomSampler(train_dataset)
        return DataLoaderWithTaskname(
            task_name,
            DataLoader(
                train_dataset,
                batch_size=self.args.per_device_train_batch_size,
                sampler=sampler,
                collate_fn=self.data_collator,  # callable collator
            ),
        )

    def get_train_dataloader(self):
        return MultitaskDataloader({
            t: self.get_single_train_dataloader(t, ds)
            for t, ds in self.train_dataset.items()
        })

    def get_eval_dataloader(self, eval_dataset=None):
        # We won't use this for per-task eval, but keep it safe/non-DDP
        eval_dataset = eval_dataset if eval_dataset is not None else self.eval_dataset
        if eval_dataset is None:
            raise ValueError("Trainer: evaluation requires an eval_dataset.")
        sampler = DistributedSampler(eval_dataset, shuffle=False) if _dist_ready() else SequentialSampler(eval_dataset)
        return DataLoader(
            eval_dataset,
            sampler=sampler,
            batch_size=self.args.per_device_eval_batch_size,
            collate_fn=self.data_collator,
        )

    def get_test_dataloader(self, test_dataset=None):
        test_dataset = test_dataset if test_dataset is not None else getattr(self, "test_dataset", None)
        if test_dataset is None:
            raise ValueError("Trainer: testing requires a test_dataset.")

        if isinstance(test_dataset, dict):
            dls = {
                t: DataLoaderWithTaskname(
                    t,
                    DataLoader(
                        ds,
                        batch_size=self.args.per_device_eval_batch_size,
                        sampler=SequentialSampler(ds),
                        collate_fn=self.data_collator,
                    )
                )
                for t, ds in test_dataset.items()
            }
            return dls
        else:
            return DataLoader(
                test_dataset,
                batch_size=self.args.per_device_eval_batch_size,
                sampler=SequentialSampler(test_dataset),
                collate_fn=self.data_collator,
            )

    @torch.no_grad()
    def predict_per_task(self, test_datasets_dict):
        results = {}
        self.model.eval()
        for task, ds in test_datasets_dict.items():
            dl = DataLoaderWithTaskname(
                task,
                DataLoader(
                    ds,
                    batch_size=self.args.per_device_eval_batch_size,
                    sampler=SequentialSampler(ds),
                    collate_fn=self.data_collator,
                )
            )
            preds = []
            for batch in dl:
                batch = {k: v.to(self.args.device) if hasattr(v, "to") else v for k, v in batch.items()}
                outputs = self.model(**batch)
                logits = outputs.logits.detach().cpu().numpy()
                preds.append(logits)
            preds = np.concatenate(preds, axis=0)
            results[task] = (preds, {})
        return results


In [ ]:
train_args = TrainingArguments(
    output_dir="/content/mtl_xlmr",
    overwrite_output_dir=True,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.01,
    fp16=True,   
    seed=42,               
    logging_steps=50,           # we evaluate manually
    save_strategy="epoch",      # you can keep epoch saving
    save_safetensors=False,     # important for shared tensors
    report_to="none",
)

In [17]:
trainer = MultitaskTrainer(
    model=multitask_model,
    args=train_args,
    train_dataset={t: features[t]["train"] for t in TASKS},  # dict for multitask
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.test_dataset = {t: features[t]["test"] for t in TASKS}

trainer


/tmp/ipython-input-4240336984.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `MultitaskTrainer.__init__`. Use `processing_class` instead.
  trainer = MultitaskTrainer(


In [18]:
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 2, 'bos_token_id': 0, 'pad_token_id': 1}.


Step,Training Loss
50,1.311400
100,1.138500
150,1.106100
200,1.085700
250,1.010100
300,1.044600
350,1.010000
400,1.054900
450,1.108200
500,0.981800


TrainOutput(global_step=19989, training_loss=0.6756042036002797, metrics={'train_runtime': 3038.3545, 'train_samples_per_second': 105.221, 'train_steps_per_second': 6.579, 'total_flos': 1.4830546023677664e+16, 'train_loss': 0.6756042036002797, 'epoch': 3.0})

In [19]:
acc = evaluate.load("accuracy")
f1  = evaluate.load("f1")

def evaluate_task(task: str):
    ds = features[task]["validation"]
    # Build a plain DataLoader first, then wrap to inject task_name
    sampler = SequentialSampler(ds)
    base_dl = DataLoader(
        ds,
        sampler=sampler,
        batch_size=train_args.per_device_eval_batch_size,
        collate_fn=data_collator,
    )
    dl = DataLoaderWithTaskname(task, base_dl)

    all_logits, all_labels = [], []
    model = trainer.model
    model.eval()
    model.to(device)

    with torch.no_grad():
        for batch in dl:
            labels = batch.pop("labels").to(device)
            task_name = batch.pop("task_name")  # StrIgnoreDevice
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(task_name=task_name, **batch)
            logits = outputs.logits
            all_logits.append(logits.detach().cpu())
            all_labels.append(labels.detach().cpu())

    logits = torch.cat(all_logits, dim=0).numpy()
    labels = torch.cat(all_labels, dim=0).numpy()
    preds = logits.argmax(axis=-1)

    return {
        "accuracy": acc.compute(predictions=preds, references=labels)["accuracy"],
        "f1_micro": f1.compute(predictions=preds, references=labels, average="micro")["f1"],
        "f1_macro": f1.compute(predictions=preds, references=labels, average="macro")["f1"],
    }

results = {t: evaluate_task(t) for t in TASKS}
results

{'hate_type': {'accuracy': 0.7229299363057324,
  'f1_micro': 0.7229299363057324,
  'f1_macro': 0.5434855131754995},
 'hate_severity': {'accuracy': 0.7193471337579618,
  'f1_micro': 0.7193471337579618,
  'f1_macro': 0.5947359655478418},
 'to_whom': {'accuracy': 0.7109872611464968,
  'f1_micro': 0.7109872611464968,
  'f1_macro': 0.5626781836308291}}

In [24]:
import numpy as np
import pandas as pd
import torch

# === 1) Run inference per task on the test set ===
pred_logits = trainer.predict_per_task(trainer.test_dataset)  # {task: (logits, {})}

# === 2) Convert logits → label strings ===
pred_labels = {}
for task in TASKS:
    logits, _ = pred_logits[task]
    y_hat = np.argmax(logits, axis=1).tolist()
    id2label = label_maps[task]["id2label"]
    pred_labels[task] = [id2label[i] for i in y_hat]

# === 3) Load test IDs ===
test_df = pd.read_csv(TEST_TSV, sep="\t", dtype=str)

# === 4) Build submission DataFrame (with model column) ===
sub_df = pd.DataFrame({
    "id": test_df["id"],
    "hate_type": pred_labels["hate_type"],
    "hate_severity": pred_labels["hate_severity"],
    "to_whom": pred_labels["to_whom"],
    "model": model_name,   # use your model name / tag here
})

# === 5) Replace NaN / blank cells with "None" ===
sub_df = sub_df.fillna("None").replace(r"^\s*$", "None", regex=True)

# === 6) Save to TSV ===
out_path = "/content/subtask_1C.tsv"
sub_df.to_csv(out_path, sep="\t", index=False)

print(f"✅ Submission file saved: {out_path}")
print(sub_df.head())


✅ Submission file saved: /content/subtask_1C.tsv
       id       hate_type   hate_severity    to_whom             model
0   12764         Abusive            Mild    Society  xlm-roberta-base
1  202933            None  Little to None       None  xlm-roberta-base
2  165894         Abusive  Little to None       None  xlm-roberta-base
3  124999         Profane          Severe  Community  xlm-roberta-base
4  535301  Religious Hate          Severe  Community  xlm-roberta-base


In [25]:
!zip subtask_1C.zip subtask_1C.tsv

  adding: subtask_1C.tsv (deflated 89%)
